# GCcare NIST RAG API 테스트 노트북

이 노트북은 `FastAPI + LangServe`로 띄운 **/health**, **/rag** 엔드포인트를 간단히 테스트하기 위한 예시입니다.

## 0. 서버 실행 (터미널에서)

아나콘다 / venv를 활성화한 뒤, 프로젝트 루트(예: `.../gccare-langchain-rag-ai`)에서 아래 명령어를 **터미널에서** 실행해 FastAPI 서버를 먼저 띄워주세요.

```bash
python -m uvicorn api.serve_app:app --host 127.0.0.1 --port 8000 --reload
```

- 서버가 정상 실행되면: `http://127.0.0.1:8000/docs` 로도 접속 가능
- 이 노트북은 서버가 떠 있다는 가정하에 `http://127.0.0.1:8000` 으로 요청을 보냅니다.


In [3]:
import json
import textwrap
import requests

BASE_URL = "http://127.0.0.1:8000"

print("BASE_URL:", BASE_URL)

BASE_URL: http://127.0.0.1:8000


## 1. `/health` 엔드포인트 확인

FastAPI 서버가 제대로 떠 있는지 헬스 체크를 해봅니다.


In [4]:
health_res = requests.get(f"{BASE_URL}/health")
print("Status code:", health_res.status_code)
print("Response JSON:")
print(health_res.json())

Status code: 200
Response JSON:
{'status': 'ok'}


## 2. RAG `/rag/invoke` 기본 헬퍼 함수

LangServe가 마운트한 `/rag/invoke` 엔드포인트에 POST 요청을 보내는 헬퍼 함수를 만듭니다.

- body 형식: `{"input": "<질문 문자열>"}`


In [13]:
def ask_rag(question: str):
    """RAG 체인(/rag/invoke)에 질문을 던지고 결과를 반환하는 헬퍼 함수."""
    url = f"{BASE_URL}/rag/invoke"
    payload = {"input": question}
    print("\n=== 질문 =============================")
    print(question)
    print("=====================================")
    
    res = requests.post(url, json=payload)
    print("Status code:", res.status_code)
    try:
        data = res.json()
        print("\n원시 응답(JSON):")
        print(json.dumps(data, ensure_ascii=False, indent=2))
    except Exception:
        print("\n응답 본문(text):")
        print(res.text)
        
    return res

## 3. 기본 테스트 질문 1 — NIST AI RMF 목적

가장 기본적인 질문으로, NIST AI Risk Management Framework의 목적에 대해 물어봅니다.


In [14]:
_ = ask_rag("NIST AI Risk Management Framework의 핵심 목적이 뭐야?")


=== 질문 =============================
NIST AI Risk Management Framework의 핵심 목적이 뭐야?
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI Risk Management Framework의 핵심 목적은 인공지능 시스템의 위험을 관리하고 최소화하여, 안전하고 신뢰할 수 있는 AI 기술을 개발하고 사용하는 데 기여하는 것입니다.",
  "metadata": {
    "run_id": "739cf058-a8ed-4be8-af48-3c31d1bc9d28",
    "feedback_tokens": []
  }
}


## 4. 테스트 질문 2 — Governance 관련

거버넌스(거버넌스 기능, 역할 등)에 초점을 맞춰서 질문해 봅니다.


In [15]:
_ = ask_rag("NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.")


=== 질문 =============================
NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF(인공지능 리스크 관리 프레임워크)에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할에는 다음과 같은 것들이 포함됩니다:\n\n1. **정책 수립**: 인공지능 사용에 대한 명확한 정책과 기준을 설정합니다.\n2. **책임 분담**: 인공지능 시스템의 개발 및 운영에 관련된 책임과 역할을 명확히 정의합니다.\n3. **위험 관리**: 인공지능 시스템에서 발생할 수 있는 위험을 식별하고 평가하여 관리하는 절차를 수립합니다.\n4. **모니터링 및 평가**: 시스템의 성능과 영향력을 지속적으로 평가하고 모니터링합니다.\n5. **이해관계자 참여**: 다양한 이해관계자와의 소통을 통해 의견을 반영하고 투명성을 높입니다.\n\n이러한 활동을 통해 조화롭고 책임감 있는 인공지능 운영을 도모합니다.",
  "metadata": {
    "run_id": "d42d52fb-7bd7-4bf9-9700-9317423bd18f",
    "feedback_tokens": []
  }
}


## 5. 테스트 질문 3 — 라이프사이클 관점 질문

NIST AI RMF의 구조(예: Map, Measure, Manage, Govern 등)에 초점.


In [16]:
_ = ask_rag("NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.")


=== 질문 =============================
NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF의 각 단계는 다음과 같습니다:\n\n1. **Map**: 인공지능 시스템의 목표와 위험을 식별하고 이해하는 단계로, 시스템의 맥락과 환경을 명확히 하는 과정입니다.\n\n2. **Measure**: AI 시스템의 성능 및 위험을 평가하고 측정하여, 시스템이 목표를 달성하고 있는지 검토하는 단계입니다.\n\n3. **Manage**: 식별된 위험을 관리하고 완화하기 위한 전략과 조치를 개발하는 단계로, 위험을 효과적으로 통제하는 방법을 모색합니다.\n\n4. **Govern**: AI 시스템의 관리 및 감독 체계를 구축하고, 관련 정책과 절차를 제정하여 지속적으로 시스템을 책임지고 운영하는 단계입니다.",
  "metadata": {
    "run_id": "22688888-1032-4189-8661-8b49e3248a4a",
    "feedback_tokens": []
  }
}


## 6. 테스트 질문 4 — 실무 적용 관점 질문

실제로 조직이 AI 리스크 관리를 도입할 때의 적용 관점 질문입니다.


In [17]:
_ = ask_rag(
    "중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?"
)


=== 질문 =============================
중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF를 참고하여 생성형 AI 서비스 리스크 관리를 시작할 때 우선적으로 점검해야 할 영역은 다음과 같습니다:\n\n1. **목표 설정 및 이해**: 서비스의 목표와 기능을 명확히 정의하고, 이러한 목표가 사용자 및 이해관계자에게 미치는 영향을 평가합니다.\n2. **데이터 관리**: 데이터 출처, 품질, 편향성 및 개인정보 보호 문제를 점검하여 데이터 관리의 신뢰성을 확보합니다.\n3. **모델의 안전성 및 신뢰성**: AI 모델의 성능을 테스트하고, 예상치 못한 결과나 오류를 방지하기 위한 안전성 검토를 진행합니다.\n4. **편향 및 차별성 평가**: 모델이 특정 그룹에 대해 편향된 결과를 초래하지 않도록 편향 분석을 수행합니다.\n5. **윤리적 고려사항**: AI 서비스가 법적 및 윤리적 기준을 준수하고 있는지 점검합니다.\n\n이 단계들을 통해 리스크 관리의 기초를 다질 수 있습니다.",
  "metadata": {
    "run_id": "bf61ec91-f0bd-4375-ad00-791cb003d642",
    "feedback_tokens": []
  }
}


## 7. 테스트 질문 5 — 문서에 없을 법한 질문 (Out-of-Scope)

RAG가 **컨텍스트에 없는 내용은 모른다고 답하는지** 확인하기 위해 일부러 문서와 관계없는 질문을 던져봅니다.


In [18]:
_ = ask_rag("NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?")


=== 질문 =============================
NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?
Status code: 200

원시 응답(JSON):
{
  "output": "모른다.",
  "metadata": {
    "run_id": "6cfd5ca6-a276-4ce5-b3ca-c8518f5cc42d",
    "feedback_tokens": []
  }
}


## 8. 자유 질문 셀

원하는 질문으로 직접 RAG 호출을 해볼 수 있는 셀입니다. `question` 변수만 바꿔서 실행하세요.


In [19]:
question = "AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘."
_ = ask_rag(question)


=== 질문 =============================
AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF(Artificial Intelligence Risk Management Framework)에서 제안하는 원칙들은 다음과 같습니다:\n\n1. **신뢰성**: AI 시스템이 일관되게 신뢰할 수 있는 결과를 제공해야 합니다.\n2. **투명성**: AI의 의사 결정 과정이 외부인에게 이해 가능해야 합니다.\n3. **공정성**: AI 시스템이 편향되지 않고 모든 사용자에게 공정하게 작동해야 합니다.\n4. **프라이버시 보호**: 개인 정보와 데이터가 안전하게 보호되어야 합니다.\n5. **책임성**: AI 시스템의 결정과 행동에 대해 책임이 명확히 규명되어야 합니다.\n\n이 원칙들은 AI 시스템의 신뢰성과 투명성을 동시에 강화하는 데 도움을 줍니다.",
  "metadata": {
    "run_id": "5d04d387-b547-4290-a052-6038d89cb73c",
    "feedback_tokens": []
  }
}
